In [1]:
import random
from firebase_admin import initialize_app, credentials, db

In [2]:
import json

In [3]:
cred = credentials.Certificate('creds.json')

In [4]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [5]:
seed = random.randrange(2 ** 32)
seed

2252495782

In [6]:
players = ["Aang", "Katara", "Sokka"]
#random.shuffle(players)

In [7]:
with open("board.json") as json_file:
    board = json.load(json_file)

In [8]:
db.reference("test").set({
    'seed': seed,
    'players': players,
    'board': board,
})

In [12]:
', '.join([str(i) for i in sorted(a+b+2 for a in range(6) for b in range(6))])

'2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 11, 11, 12'